In [8]:
#https://www.kaggle.com/kshitijmohan/lstm-stock-prediction
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy import stats
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential, load_model, save_model
import seaborn as sns
from pylab import rcParams
from sklearn.preprocessing import MinMaxScaler

from sklearn import metrics
import pydot as pyd
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import tabpy_client
import requests
from pickle import dump
import base64
import json

import json
#%matplotlib inline

sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 14, 8

RANDOM_SEED = 42

df = pd.read_csv("Data/prices-split-adjusted.csv")
df.info()


plot_x = df['date'].copy()
df.set_index("date", inplace = True)
df.index = pd.to_datetime(df.index)
df.head()


google_stock = df[df['symbol'] == 'GOOG']
google_stock.head()

x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()
goog_df = google_stock.copy()
goog_df.drop(['symbol'], axis=1, inplace=True)
x = goog_df[['open', 'low', 'high', 'volume']].copy()

y = goog_df['close'].copy()

x[['open', 'low', 'high', 'volume']] = x_scaler.fit_transform(x)
y = y_scaler.fit_transform(y.values.reshape(-1, 1))




model = load_model("c:\TNB\Learning\Python\SQL Server\model_01.h5")

#SVG(model_to_dot(model,  show_shapes=False, show_layer_names=True, rankdir='HB').create(prog='dot', format='svg'))
model.summary()

dump(x_scaler,open('x_scaler.pkl','wb'))
dump(y_scaler,open('y_scaler.pkl','wb'))

def Encode(lst):

    res = ''
    #list to json string
    res = json.dumps(lst)

    #Encode string to base64

    sample_string = res
    sample_string_bytes = sample_string.encode("ascii")

    base64_bytes = base64.b64encode(sample_string_bytes)
    base64_string = base64_bytes.decode("ascii")

    return base64_string

def StockPricePredictApi(open,low,high,volumn):
    print('StockPricePredict aaaa')
    from keras.models import Sequential, load_model, save_model
    import tensorflow as tf
    graph = tf.get_default_graph()
    print('open')
    print(open)
    
    print('low')
    print(low)
    
    print('high')
    print(high)
    
    print('volumn')
    print(volumn)
    #print(x_scaler.transform(volumn))
    
    
    a = pd.DataFrame(list(zip(open, low,high,volumn,open)), columns = ['open', 'low', 'high', 'volume','close']) 
    
    Y = a[['close']]
    X = a[['open', 'low', 'high', 'volume']]
    
    X[['open', 'low', 'high', 'volume']] = x_scaler.transform(X)
    Y = y_scaler.transform(Y)
    
    X['close'] = Y
    
    
    X = X.values
    
    #print('X.values')
    
    #print(X)
    
    #print(type(X))
    
    res = []
    
    print('zzzzzzzzzzzzz')
    i = 0
    url = 'http://localhost:1234/getstock02'
    for z in np.nditer(X, order='F'):
        print(z, end=' ')
        url = url + '/'+str(z)   
        i = i+1
        if (i%5==0):
            print(i)
            print(requests.get(url).text)
            res.append([float(requests.get(url).text)])
            url = 'http://localhost:1234/getstock02'
    print('zzzzzzzzzzzzz')    
    
    X = np.reshape(X,(X.shape[0],1,X.shape[1]))
    
    print(X)
    
    print(type(X))
    
    print('res')
    print(res)
    print(type(res))
    
    print(np.array(res))
    
    res = np.array(res)
    res = y_scaler.inverse_transform(res)
    print('new res')
    
    #res = list(res)
    res1 = []
    for t in np.nditer(res, order='F'):
        res1.append(t.item(0))
        
    print(res)
    print(type(res))
    
    print(type(open))

    
    print(res1)
    print(type(res1))
    
    print('open')
    print(open)
    
    
    #model1 = load_model("c:\TNB\Learning\Python\SQL Server\model_01.h5")

    #with graph.as_default():
    #   res = model1.predict(X)
    #   print('aaaaa')
    #   print(res)
    #print(model.predict(X))
    
    #print(type(model.predict(X)))
    
    #ttt = y_scaler.inverse_transform(model.predict(X))
    #print(ttt)
    
    #print(type(ttt))
    
    return res1
   
vClose = StockPricePredictApi([312.304948,312.304948] , [310.955001,310.955001] , [313.580158,313.580158] , [3927000.0,3927000.0])    

print('vClose Api')
print(vClose)


def StockPricePredictApiOneCall(open,low,high,volumn):
    print('StockPricePredict aaaa')
    from keras.models import Sequential, load_model, save_model
    import tensorflow as tf
    graph = tf.get_default_graph()
    print('open')
    print(open)
    
    print('low')
    print(low)
    
    print('high')
    print(high)
    
    print('volumn')
    print(volumn)
    #print(x_scaler.transform(volumn))
    
    
    url = 'http://localhost:1234/getstock03'
    url= url +'/'+ Encode(open)
    url= url +'/'+ Encode(low)
    url= url +'/'+ Encode(high)
    url= url +'/'+ Encode(volumn)
    url= url +'/'+ Encode(open)
    
    print(url)
    
    jsonString = requests.get(url).text
    
    res = json.loads(jsonString)
      
    return res
   
vCloseOneCall = StockPricePredictApiOneCall([312.304948,312.304948] , [310.955001,310.955001] , [313.580158,313.580158] , [3927000.0,3927000.0])    

print('vCloseOneCall Api')
print(vCloseOneCall)
print(type(vCloseOneCall))



# Connect to TabPy server using the client library
connection = tabpy_client.Client('http://localhost:9004/')

# Publish the SuggestDiagnosis function to TabPy server so it can be used from Tableau
# Using the name DiagnosticsDemo and a short description of what it does
connection.deploy('StockPricePredictApi',
                  StockPricePredictApi,
                  'Returns StockPricePredict', override = True)

connection.deploy('StockPricePredictApiOneCall',
                  StockPricePredictApiOneCall,
                  'Returns StockPricePredictApiOneCall', override = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1, 128)            68608     
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_______

C:\TNB\Learning\Python\MachineLearning\Demo\venv\lib\site-packages\ipykernel_launcher.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\TNB\Learning\Python\MachineLearning\Demo\venv\lib\site-packages\pandas\core\frame.py:3493: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.loc._setitem_with_indexer((slice(None), indexer), value)
C:\TNB\Learning\Python\MachineLearning\Demo\venv\lib\site-packages\pandas\core\frame.py:3464: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://